### 5. Smart Charging Using Reinforcement Learning:
**Original Exercise:** <br>
Consider an electric taxi driver who can charge her vehicle at home. To simplify the problem, we assume that the vehicle always arrives at home at 2 p.m. and leaves the garage at 4 p.m. each day. We want to design an intelligent charging system (an automated agent). Therefore, instead of a flat charging rate, the charging agent adjusts the charging power every 15 minutes, which is bounded between 0 kW and the highest rate (e.g., 22 kW). Also, the vehicle's battery has a capacity that cannot be exceeded. After leaving the garage, the taxi needs enough energy to complete its working day. The energy demand is a stochastic value following a normal distribution (you should choose the parameters, e.g., 𝜇= 30 kWh, 𝜎 = 5 kWh) and must be generated exactly when the driver wants to leave. The agent’s goal is to avoid running out of energy (you should consider a very high penalty for running out of energy) and to minimize the recharging cost. The recharging cost follows an exponential function of the power (i.e., ), where 𝛼𝑡 is the time coefficient and p is the charging rate.

The task is to create the environment (a very simple discrete event simulation) that receives the agent's decisions and returns the reward. In addition, you must define a Markov decision process, including states, actions, and reward function, and solve it using a reinforcement learning algorithm (e.g., deep q-network) to find optimal charging policies. To allow the use of discrete action methods, you can consider only limited charging options such as zero, low, medium, high.


**In Bulletpoints:**
- Problem description:
    - An electric taxi driver can charge her vehicle at home between 2 p.m. and 4 p.m. each day
    - The charging agent adjusts the charging power every 15 minutes within a range of 0 kW to 22 kW
    - The vehicle's battery has a limited capacity that cannot be exceeded
    - The taxi needs enough energy to complete its working day, which is a random value following a normal distribution (e.g., 𝜇= 30 kWh, 𝜎 = 5 kWh)
    - The agent’s goal is to avoid running out of energy (with a very high penalty) and to minimize the recharging cost, which is an exponential function of the power (i.e., ), where 𝛼𝑡 is the time coefficient and p is the charging rate
- Task description:
    - Create the environment that simulates the charging process and the energy demand, and returns the reward to the agent based on its actions
    - Define a Markov decision process, including states, actions, and reward function, that models the problem
    - Solve the Markov decision process using a reinforcement learning algorithm (e.g., deep q-network) to find optimal charging policies
    - Consider only discrete action methods, such as zero, low, medium, high, for the charging power

In [1]:
### Imports
#Basic
import numpy as np
import random
import math
# Gym
from gym import Env
from gym.spaces import Box, Discrete
# Keras
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
# Keras RL
from rl.agents import DQNAgent
from rl.agents import SARSAAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

## Simple Environment for Testing

In [25]:
### Simple environment without probability for testing
class SimpleEnvironment(Env):
    def __init__(self):
        #Possible Actions for charging zero, low, medium to high
        self.action_space = Discrete(4)
        #Vehicle's battery: 69KWh; Timeframe 2p.m. to 4p.m.: 8x 15 minute intervals
        self.battery_limit = 69
        self.observation_space = np.array([Box(low=np.array([0]), high = np.array([self.battery_limit])), Box(low=np.array([0]), high = np.array([8]))])
        # Starting at 2p.m.: 0 (going to 3:45p.m.: 7)
        self.time = 0
        self.time_delta =  15/60
        # 20 KWh loaded battery at initialization
        self.battery = 20
        # our state consisting of battery status and time
        self.state = np.array([self.battery, self.time])


    def step(self, action):
        # Setting loading interval +1/8 /--> +15/120 minutes
        self.time += 1
        self.state[1] = self.time

        # Seting new battery state
        #zero
        load = 0
        if action == 2:
            #low
            load += 7 * self.time_delta
            # loading until max capacity 
            if self.battery + load > self.battery_limit:
                diff = self.battery_limit - self.battery
                load = diff / self.time_delta
        if action == 3:
            #medium
           load += 14 * self.time_delta
            # loading until max capacity 
           if self.battery + load > self.battery_limit:
                diff = self.battery_limit - self.battery
                load = diff / self.time_delta
        if action == 4:
            #high
            load += 22 * self.time_delta
            # loading until max capacity 
            if self.battery + load > self.battery_limit:
                diff = self.battery_limit - self.battery
                load = diff / self.time_delta
        # load multiplied by 15 min to get the 
        self.battery += load
        self.state[0] = self.battery

        # Simple cost function
        reward = (1/self.time)*load*(-1)

        
        print("time:" + str(self.time) + " |load:" + str(load) + " |reward:" + str(round(reward, 2)))

        #Checking if 2 Hours are done
        if self.time >= 8:
            # Static Demand
            kwh_needed = 30
            print("|needed:" + str(round(kwh_needed, 2))+" |battery:" + str(round(self.battery, 2)))
            # The agent’s goal is to avoid running out of energy (with a very high penalty) 
            if kwh_needed > self.battery:
                reward -= 25
            done = True
        else:
            done = False

        info = {}

        # Returning the step information
        return self.state, reward, done, info
    
    def reset(self):
        # Starting at 2p.m.: 0 (going to 3:45p.m.: 7)
        self.time = 0
        # 20 KWh loaded battery at initialization
        self.battery = 20
        # our state consisting of battery status and time
        self.state = np.array([self.battery, self.time])
        return self.state

In [26]:
simpleEnv = SimpleEnvironment()

In [27]:
# Some random examples
episodes = 2
for episode in range(1, episodes+1):
    print("__ Day " + str(episode) + " ___")
    state = simpleEnv.reset()
    done = False
    score = 0 
    while not done:
        action = simpleEnv.action_space.sample()
        n_state, reward, done, info = simpleEnv.step(action)
        score+=reward
    print('--> Score:{}'.format(round(score, 2)))

__ Day 1 ___
time:1 |load:0 |reward:-0.0
time:2 |load:0 |reward:-0.0
time:3 |load:3.5 |reward:-1.17
time:4 |load:0 |reward:-0.0
time:5 |load:3.5 |reward:-0.7
time:6 |load:0 |reward:-0.0
time:7 |load:0 |reward:-0.0
time:8 |load:3.5 |reward:-0.44
|needed:30 |battery:30.5
--> Score:-2.3
__ Day 2 ___
time:1 |load:1.75 |reward:-1.75
time:2 |load:1.75 |reward:-0.88
time:3 |load:1.75 |reward:-0.58
time:4 |load:1.75 |reward:-0.44
time:5 |load:0 |reward:-0.0
time:6 |load:3.5 |reward:-0.58
time:7 |load:0 |reward:-0.0
time:8 |load:1.75 |reward:-0.22
|needed:30 |battery:32.25
--> Score:-4.45


In [28]:
# Destilling Information from hour model
states = simpleEnv.observation_space.shape
actions = simpleEnv.action_space.n
print("Actions: " + str(actions) + " | States: " + str(states))

Actions: 4 | States: (2,)


In [29]:
# Defining our model
model = Sequential()    
model.add(Flatten(input_shape=(1,2)))
model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(actions, activation='linear'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 2)                 0         
                                                                 
 dense_8 (Dense)             (None, 8)                 24        
                                                                 
 dense_9 (Dense)             (None, 4)                 36        
                                                                 
 dense_10 (Dense)            (None, 4)                 20        
                                                                 
Total params: 80
Trainable params: 80
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Defining and training of Deep Q-Network Agent
policy = BoltzmannQPolicy()
memory = SequentialMemory(limit=100, window_length=1)
dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=8*100, target_model_update=1e-2)
dqn.compile(Adam(learning_rate=0.01), metrics=['mae'])
dqn.fit(simpleEnv, nb_steps=40000, visualize=False, verbose=1)

Training for 40000 steps ...
Interval 1 (0 steps performed)
time:1 |load:0 |reward:-0.0
    1/10000 [..............................] - ETA: 17:30 - reward: 0.0000e+00time:2 |load:0 |reward:-0.0
time:3 |load:0 |reward:-0.0
time:4 |load:0 |reward:-0.0
time:5 |load:0 |reward:-0.0
time:6 |load:0 |reward:-0.0
time:7 |load:0 |reward:-0.0
time:8 |load:0 |reward:-0.0
|needed:30 |battery:20
time:1 |load:0 |reward:-0.0
time:2 |load:0 |reward:-0.0
time:3 |load:0 |reward:-0.0
time:4 |load:0 |reward:-0.0
time:5 |load:0 |reward:-0.0
time:6 |load:0 |reward:-0.0
time:7 |load:0 |reward:-0.0
time:8 |load:0 |reward:-0.0
|needed:30 |battery:20
time:1 |load:0 |reward:-0.0
time:2 |load:0 |reward:-0.0
time:3 |load:0 |reward:-0.0
time:4 |load:0 |reward:-0.0
time:5 |load:0 |reward:-0.0
time:6 |load:0 |reward:-0.0
time:7 |load:0 |reward:-0.0
time:8 |load:0 |reward:-0.0
|needed:30 |battery:20
time:1 |load:0 |reward:-0.0
time:2 |load:0 |reward:-0.0
time:3 |load:0 |reward:-0.0
time:4 |load:0 |reward:-0.0
time:5 |l

c:\Users\NickPC\.virtualenvs\AAA_2023-67gSWIjI\lib\site-packages\keras\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


time:1 |load:0 |reward:-0.0
time:2 |load:0 |reward:-0.0
time:3 |load:0 |reward:-0.0
time:4 |load:0 |reward:-0.0
time:5 |load:0 |reward:-0.0
time:6 |load:0 |reward:-0.0
time:7 |load:0 |reward:-0.0
time:8 |load:0 |reward:-0.0
|needed:30 |battery:20
time:1 |load:0 |reward:-0.0
time:2 |load:0 |reward:-0.0
time:3 |load:0 |reward:-0.0
time:4 |load:0 |reward:-0.0
time:5 |load:0 |reward:-0.0
time:6 |load:0 |reward:-0.0
time:7 |load:0 |reward:-0.0
time:8 |load:0 |reward:-0.0
|needed:30 |battery:20
time:1 |load:0 |reward:-0.0
time:2 |load:0 |reward:-0.0
time:3 |load:0 |reward:-0.0
time:4 |load:0 |reward:-0.0
time:5 |load:0 |reward:-0.0
time:6 |load:0 |reward:-0.0
time:7 |load:0 |reward:-0.0
time:8 |load:0 |reward:-0.0
|needed:30 |battery:20
time:1 |load:0 |reward:-0.0
time:2 |load:0 |reward:-0.0
time:3 |load:0 |reward:-0.0
time:4 |load:0 |reward:-0.0
time:5 |load:0 |reward:-0.0
time:6 |load:0 |reward:-0.0
time:7 |load:0 |reward:-0.0
time:8 |load:0 |reward:-0.0
|needed:30 |battery:20
time:1 |load

In [43]:
# Testing the agent on the simple Environment
results = dqn.test(simpleEnv, nb_episodes=2, visualize=False)
print(np.mean(results.history['episode_reward']))

Testing for 2 episodes ...
time:1 |load:0 |reward:-0.0
time:2 |load:0 |reward:-0.0
time:3 |load:0 |reward:-0.0
time:4 |load:0 |reward:-0.0
time:5 |load:0 |reward:-0.0
time:6 |load:0 |reward:-0.0
time:7 |load:0 |reward:-0.0
time:8 |load:0 |reward:-0.0
|needed:30 |battery:20
Episode 1: reward: -25.000, steps: 8
time:1 |load:0 |reward:-0.0
time:2 |load:0 |reward:-0.0
time:3 |load:0 |reward:-0.0
time:4 |load:0 |reward:-0.0
time:5 |load:0 |reward:-0.0
time:6 |load:0 |reward:-0.0
time:7 |load:0 |reward:-0.0
time:8 |load:0 |reward:-0.0
|needed:30 |battery:20
Episode 2: reward: -25.000, steps: 8
-25.0


## Environment from the Exercise

In [3]:
import tensorflow
print("Num GPUs Available: ", len(tensorflow.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [5]:
tensorflow.config.experimental.list_physical_devices('GPU')

[]

In [142]:

class Environment(Env):
    def __init__(self):
        #Possible Actions for charging zero, low, medium to high
        self.action_space = Discrete(4)
        #Vehicle's battery: 69KWh; Timeframe 2p.m. to 4p.m.: 8x 15 minute intervals
        self.battery_limit = 69
        self.observation_space = np.array([Box(low=np.array([0]), high = np.array([self.battery_limit])), Box(low=np.array([0]), high = np.array([8]))])
        # Starting at 2p.m.: 0 (going to 3:45p.m.: 7)
        self.time = 0
        self.time_delta =  15/60
        # 20 KWh loaded battery at initialization
        #self.battery = 10 + random.randint(-10,10)
        self.battery = 21
        # our state consisting of battery status and time
        self.state = np.array([self.battery, self.time])


    def step(self, action):
        # Setting loading interval +1/8 /--> +15/120 minutes
        self.time += 1
        self.state[1] = self.time

        # Seting new battery state
        #zero
        load = 0
        if action == 2:
            #low
            load += 7 * self.time_delta
            # loading until max capacity 
            if self.battery + load > self.battery_limit:
                diff = self.battery_limit - self.battery
                load = diff / self.time_delta
        if action == 3:
            #medium
            load += 14 * self.time_delta
            # loading until max capacity 
            if self.battery + load > self.battery_limit:
                diff = self.battery_limit - self.battery
                load = diff / self.time_delta
        if action == 4:
            #high
            load += 22 * self.time_delta
            # loading until max capacity 
            if self.battery + load > self.battery_limit:
                diff = self.battery_limit - self.battery
                load = diff / self.time_delta
        # load multiplied by 15 min to get the 
        self.battery += load
        self.state[0] = self.battery

        # Cost function
        reward = self.time * math.exp(load) * (-1)
        # Because e^0 = 1
        if action == 0:
            reward = 0

        
       # print("time:" + str(self.time) + " |load:" + str(load) + " |reward:" + str(round(reward, 2)))

        #Checking if 2 Hours are done
        if self.time >= 8:
            #Demand is a random value following a normal distribution (e.g., 𝜇= 30 kWh, 𝜎 = 5 kWh)
            kwh_needed = np.random.normal(loc=30, scale=5)
            #print("|needed:" + str(round(kwh_needed, 2))+" |battery:" + str(round(self.battery, 2)))
            # The agent’s goal is to avoid running out of energy (with a very high penalty) 
            if kwh_needed > self.battery:
                #print("NO BATTERY")
                reward -= 10000
            done = True
        else:
            done = False

        info = {}

        # Returning the step information
        return self.state, reward, done, info
    
    def reset(self):
        # Starting at 2p.m.: 0 (going to 3:45p.m.: 7)
        self.time = 0
        # 20 KWh loaded battery at initialization
        #self.battery = 10 + random.randint(-10,10)
        self.battery = 21
        # our state consisting of battery status and time
        self.state = np.array([self.battery, self.time])
        return self.state

In [143]:

class EnvironmentPrint(Env):
    def __init__(self):
        #Possible Actions for charging zero, low, medium to high
        self.action_space = Discrete(4)
        #Vehicle's battery: 69KWh; Timeframe 2p.m. to 4p.m.: 8x 15 minute intervals
        self.battery_limit = 69
        self.observation_space = np.array([Box(low=np.array([0]), high = np.array([self.battery_limit])), Box(low=np.array([0]), high = np.array([8]))])
        # Starting at 2p.m.: 0 (going to 3:45p.m.: 7)
        self.time = 0
        self.time_delta =  15/60
        # 20 KWh loaded battery at initialization
        #self.battery = 10 + random.randint(-10,10)
        self.battery = 21
        # our state consisting of battery status and time
        self.state = np.array([self.battery, self.time])


    def step(self, action):
        # Setting loading interval +1/8 /--> +15/120 minutes
        self.time += 1
        self.state[1] = self.time

        # Seting new battery state
        #zero
        load = 0
        if action == 2:
            #low
            load += 7 * self.time_delta
            # loading until max capacity 
            if self.battery + load > self.battery_limit:
                diff = self.battery_limit - self.battery
                load = diff / self.time_delta
        if action == 3:
            #medium
            load += 14 * self.time_delta
            # loading until max capacity 
            if self.battery + load > self.battery_limit:
                diff = self.battery_limit - self.battery
                load = diff / self.time_delta
        if action == 4:
            #high
            load += 22 * self.time_delta
            # loading until max capacity 
            if self.battery + load > self.battery_limit:
                diff = self.battery_limit - self.battery
                load = diff / self.time_delta
        # load multiplied by 15 min to get the 
        self.battery += load
        self.state[0] = self.battery

        # Cost function
        reward = self.time * math.exp(load) * (-1)
        # Because e^0 = 1
        if action == 0:
            reward = 0

        
        print("time:" + str(self.time) + " |load:" + str(load) + " |reward:" + str(round(reward, 2)))

        #Checking if 2 Hours are done
        if self.time >= 8:
            #Demand is a random value following a normal distribution (e.g., 𝜇= 30 kWh, 𝜎 = 5 kWh)
            kwh_needed = np.random.normal(loc=30, scale=5)
            print("|needed:" + str(round(kwh_needed, 2))+" |battery:" + str(round(self.battery, 2)))
            # The agent’s goal is to avoid running out of energy (with a very high penalty) 
            if kwh_needed > self.battery:
                #print("NO BATTERY")
                reward -= 10000
            done = True
        else:
            done = False

        info = {}

        # Returning the step information
        return self.state, reward, done, info
    
    def reset(self):
        # Starting at 2p.m.: 0 (going to 3:45p.m.: 7)
        self.time = 0
        # 20 KWh loaded battery at initialization
        #self.battery = 10 + random.randint(-10,10)
        self.battery = 21
        # our state consisting of battery status and time
        self.state = np.array([self.battery, self.time])
        return self.state

In [144]:
env = Environment()

In [145]:
envPrint = EnvironmentPrint()

In [146]:
# Destilling Information from hour model
states = env.observation_space.shape
actions = env.action_space.n
print("Actions: " + str(actions) + " | States: " + str(states))

Actions: 4 | States: (2,)


In [147]:
# Defining our model
model = Sequential()    
model.add(Flatten(input_shape=(1,2)))
#model.add(Dense(32, activation='relu'))
#model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(actions, activation='linear'))
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_11 (Flatten)        (None, 2)                 0         
                                                                 
 dense_39 (Dense)            (None, 8)                 24        
                                                                 
 dense_40 (Dense)            (None, 4)                 36        
                                                                 
 dense_41 (Dense)            (None, 4)                 20        
                                                                 
Total params: 80
Trainable params: 80
Non-trainable params: 0
_________________________________________________________________


In [148]:
# Defining and training of Deep Q-Network Agent
policy = BoltzmannQPolicy()
memory = SequentialMemory(limit=10000*2, window_length=1)
dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10000*1, target_model_update=1e-2)
dqn.compile(Adam(learning_rate=0.05), metrics=['mae'])
dqn.fit(env, nb_steps=10000*4, visualize=False, verbose=1)

Training for 40000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 14s 1ms/step - reward: -1216.5327
1250 episodes - episode_reward: -9732.262 [-10292.924, -28.000]

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 108s 11ms/step - reward: -333.6886
1250 episodes - episode_reward: -2669.509 [-10895.529, 0.000] - loss: 4445999.493 - mae: 330.824 - mean_q: -92.091

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 104s 10ms/step - reward: -244.4449
1250 episodes - episode_reward: -1955.559 [-10784.674, -5.755] - loss: 2021754.625 - mae: 437.468 - mean_q: -346.918

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 106s 11ms/step - reward: -526.1539
done, took 331.552 seconds


In [150]:
# Testing the agent on the simple Environment
results = dqn.test(envPrint, nb_episodes=20, visualize=False)
print(np.mean(results.history['episode_reward']))

Testing for 20 episodes ...
time:1 |load:1.75 |reward:-5.75
time:2 |load:1.75 |reward:-11.51
time:3 |load:1.75 |reward:-17.26
time:4 |load:1.75 |reward:-23.02
time:5 |load:1.75 |reward:-28.77
time:6 |load:1.75 |reward:-34.53
time:7 |load:1.75 |reward:-40.28
time:8 |load:1.75 |reward:-46.04
|needed:27.46 |battery:35.0
Episode 1: reward: -207.166, steps: 8
time:1 |load:1.75 |reward:-5.75
time:2 |load:1.75 |reward:-11.51
time:3 |load:1.75 |reward:-17.26
time:4 |load:1.75 |reward:-23.02
time:5 |load:1.75 |reward:-28.77
time:6 |load:1.75 |reward:-34.53
time:7 |load:1.75 |reward:-40.28
time:8 |load:1.75 |reward:-46.04
|needed:26.09 |battery:35.0
Episode 2: reward: -207.166, steps: 8
time:1 |load:1.75 |reward:-5.75
time:2 |load:1.75 |reward:-11.51
time:3 |load:1.75 |reward:-17.26
time:4 |load:1.75 |reward:-23.02
time:5 |load:1.75 |reward:-28.77
time:6 |load:1.75 |reward:-34.53
time:7 |load:1.75 |reward:-40.28
time:8 |load:1.75 |reward:-46.04
|needed:29.16 |battery:35.0
Episode 3: reward: -207

In [80]:
1* math.exp(22/4) * (-1)

-244.69193226422038

In [101]:
(1* math.exp(14/4) + 2* math.exp(14/4) + 3* math.exp(14/4) + 4* math.exp(14/4) + 5* math.exp(14/4) + 6* math.exp(14/4)  + 7* math.exp(14/4) + 8* math.exp(14/4))*(-1)

-1192.1562705129234

In [98]:
(14*8)/4

28.0

In [100]:
(1* math.exp(22/4) + 2* math.exp(22/4) + 3* math.exp(14/4) + 4* math.exp(14/4) + 5* math.exp(14/4) + 6* math.exp(14/4)  + 7* math.exp(14/4))*(-1)

-1561.9620957599689

In [99]:
(14*5+22*2)/4

28.5

In [131]:
1.75*8

14.0

In [49]:
1* math.exp(22/4) * (-1) + 2* math.exp(22/4) * (-1) + 4* math.exp(14/4) * (-1)

-866.5376046274304

In [71]:
np.random.normal(loc=30, scale=5)

38.50503978316501

In [77]:
3.5*8

28.0

2.8

In [89]:
28*4/22

5.090909090909091

In [121]:
math.exp(22/4)*(-1)*(1+2+3+4+5+6+7+8)

-8808.909561511933